In [22]:
import requests
import pandas as pd

# Your Polygon.io API key
api_key = '9xBs7r4Abb2xhm9iaNP0p3le6Z7zj1__'

# Ticker symbol for FormFactor, Inc.
ticker = 'FORM'

# Endpoint URL for financials
url = f'https://api.polygon.io/v2/reference/financials/{ticker}?apiKey={api_key}'

# Make the API request
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    # Extract the financials data
    financials = data.get('results', [])
    # Load into a pandas DataFrame
    df = pd.DataFrame(financials)
    # Display the DataFrame
    print(df)
else:
    print(f'Error: {response.status_code}')

    ticker period calendarDate reportPeriod     updated     dateKey  \
0     FORM     TA   2019-12-31   2019-12-28  2020-02-22  2019-12-28   
1     FORM      Y   2019-12-31   2019-12-28  2020-02-22  2020-02-21   
2     FORM      Q   2019-12-31   2019-12-28  2020-02-22  2020-02-21   
3     FORM     QA   2019-12-31   2019-12-28  2020-02-22  2019-12-28   
4     FORM     YA   2019-12-31   2019-12-28  2020-02-22  2019-12-28   
..     ...    ...          ...          ...         ...         ...   
317   FORM     QA   2001-12-31   2001-12-29  2020-02-22  2001-12-29   
318   FORM     TA   2001-12-31   2001-12-29  2020-02-22  2001-12-29   
319   FORM     YA   2001-12-31   2001-12-29  2020-02-22  2001-12-29   
320   FORM     TA   2000-12-31   2000-12-30  2020-02-22  2000-12-30   
321   FORM     YA   2000-12-31   2000-12-30  2020-02-22  2000-12-30   

     accumulatedOtherComprehensiveIncome       assets  assetsAverage  \
0                              -659000.0  839882000.0    786919250.0   
1  

In [26]:
# Convert column names to a list
columns_list = df.columns.to_list()
print(columns_list)

['ticker', 'period', 'calendarDate', 'reportPeriod', 'updated', 'dateKey', 'accumulatedOtherComprehensiveIncome', 'assets', 'assetsAverage', 'assetsCurrent', 'assetsNonCurrent', 'assetTurnover', 'bookValuePerShare', 'capitalExpenditure', 'cashAndEquivalents', 'cashAndEquivalentsUSD', 'costOfRevenue', 'consolidatedIncome', 'currentRatio', 'debtToEquityRatio', 'debt', 'debtCurrent', 'debtNonCurrent', 'debtUSD', 'deferredRevenue', 'depreciationAmortizationAndAccretion', 'deposits', 'dividendYield', 'dividendsPerBasicCommonShare', 'earningBeforeInterestTaxes', 'earningsBeforeInterestTaxesDepreciationAmortization', 'EBITDAMargin', 'earningsBeforeInterestTaxesDepreciationAmortizationUSD', 'earningBeforeInterestTaxesUSD', 'earningsBeforeTax', 'earningsPerBasicShare', 'earningsPerDilutedShare', 'earningsPerBasicShareUSD', 'shareholdersEquity', 'averageEquity', 'shareholdersEquityUSD', 'enterpriseValue', 'enterpriseValueOverEBIT', 'enterpriseValueOverEBITDA', 'freeCashFlow', 'freeCashFlowPerSha

In [36]:
df_revenue = df[['revenues']]
df_ta_revenue = df_revenue[df['period'] == 'TA']
revenue = df_ta_revenue['revenues'].head(10).tolist
last_revenue = df_ta_revenue['revenues'].iloc[-1]

In [ ]:
# Filter for 'TA' period and select 'earningBeforeInterestTaxes' (EBIT)
df_ta_ebit = df[df['period'] == 'TA'][['earningBeforeInterestTaxes']]
# Get the last EBIT value for 'TA' period
last_ebit = df_ta_ebit['earningBeforeInterestTaxes'].iloc[-1]  # Last EBIT

# Filter for 'TA' period and select 'revenues'
df_ta_revenue = df[df['period'] == 'TA'][['revenues']]
# Get the last revenue value for 'TA' period
last_revenue = df_ta_revenue['revenues'].iloc[-1]  # Last revenue

# Calculate and save EBIT margin as a variable, rounded to 2 decimal places
if last_revenue != 0:  # Check to avoid division by zero
    ebit_margin = round(last_ebit / last_revenue, 2)
else:
    ebit_margin = None  # Handle case if revenue is zero

# Display the calculated and rounded EBIT margin
print("EBIT Margin for 'TA' period (rounded to 2 decimal places):", ebit_margin)

# ebit_margin is now a rounded variable and can be used in further calculations.

EBIT Margin for 'TA' period (rounded to 2 decimal places): 0.05


In [ ]:
import numpy as np
import pandas as pd

# Sample financial data
revenue = [1000, 1100, 1210, 1331, 1464]  # in millions
ebit_margin = 0.15  # EBIT as a percentage of revenue
tax_rate = 0.25  # Corporate tax rate
depreciation = [50, 55, 60, 66, 73]  # in millions
capex = [60, 66, 73, 80, 88]  # in millions
change_in_nwc = [10, 11, 12, 13, 14]  # in millions
wacc = 0.10  # Discount rate
terminal_growth_rate = 0.03  # Terminal growth rate
shares_outstanding = 100  # in millions

# Calculate EBIT
ebit = [rev * ebit_margin for rev in revenue]

# Calculate NOPAT (Net Operating Profit After Tax)
nopat = [e * (1 - tax_rate) for e in ebit]

# Calculate Free Cash Flow (FCF)
fcf = [nopat[i] + depreciation[i] - capex[i] - change_in_nwc[i] for i in range(len(revenue))]

# Calculate Terminal Value
terminal_value = fcf[-1] * (1 + terminal_growth_rate) / (wacc - terminal_growth_rate)

# Discount FCFs to Present Value
discount_factors = [(1 + wacc) ** i for i in range(len(fcf))]  # Corrected discount factors
pv_fcf = [fcf[i] / discount_factors[i] for i in range(len(fcf))]

# Discount Terminal Value to Present Value at the end of the projection period
pv_terminal_value = terminal_value / discount_factors[-1]

# Calculate Enterprise Value (EV)
enterprise_value = sum(pv_fcf) + pv_terminal_value

# Assume net debt is zero for simplicity
net_debt = 0

# Calculate Equity Value
equity_value = enterprise_value - net_debt

# Calculate Intrinsic Value per Share
intrinsic_value_per_share = equity_value / shares_outstanding

print(f"Intrinsic Value per Share: ${intrinsic_value_per_share:.2f}")
